In [1]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, ConfusionMatrixDisplay, classification_report
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

import streamlit as st
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
def load_data():
    df = pd.read_excel("AI_CHIR_DIG_FINAL.xlsx", sheet_name="Second cleanup")
    return df

df_raw = load_data()

df_raw

,AGE_SURGERY,SEX,FHX,FHX_1DEGREE,TOBACCO,BMI,PHX_ABDOMINAL_SURGERY,PHX_PULM,PHX_CVD,ATCD_COLONO,...,INVOLVEMENT_DISTAL.MARGIN,SAFETY.MARGIN_PROXIMAL,SAFETY.MARGIN_DISTAL,INVASION_PERINEURAL,INVASION_VASC,INVASION_LYMPH,PATH_TUMOR.PERFORATION,SUPP_N0_N+,NUM_LN_ALL,NUM_LN_POS
0,61,F,0,0.0,1.0,16.23,1.0,0,1,1.0,...,1,19.0,1.0,2.0,1.0,1.0,2.0,N0,19,0.0
1,70,H,0,0.0,0.0,27.68,0.0,0,1,0.0,...,1,14.2,0.5,2.0,1.0,1.0,1.0,N1a,23,1.0
2,39,H,0,0.0,1.0,20.23,0.0,1,1,0.0,...,1,13.0,2.8,1.0,1.0,1.0,1.0,N1a,40,1.0
3,38,H,1,0.0,1.0,20.30,0.0,1,0,0.0,...,1,14.0,16.0,2.0,2.0,1.0,1.0,NaN,25,6.0
4,25,F,0,0.0,0.0,17.30,1.0,0,0,0.0,...,2,NaN,NaN,1.0,1.0,1.0,1.0,N1b,35,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223,73,H,0,0.0,0.0,NaN,0.0,0,1,0.0,...,1,5.0,11.5,1.0,2.0,2.0,1.0,N0,32,0.0
224,55,H,0,0.0,1.0,25.73,0.0,1,1,0.0,...,1,9.0,10.0,1.0,2.0,1.0,1.0,N0,32,0.0
225,58,H,0,NaN,1.0,21.60,0.0,1,1,0.0,...,1,5.8,16.5,1.0,2.0,1.0,1.0,N0,51,0.0
226,82,H,0,0.0,1.0,NaN,1.0,1,1,1.0,...,1,6.5,18.0,2.0,1.0,2.0,1.0,N0,30,0.0


In [3]:
selected_inputs = ["SEX", "FHX"]
selected_target = ["COMPLICATION"]

In [4]:
# drop the rows where value is missing in inputs and target
df_raw = df_raw.dropna(subset=selected_inputs + selected_target).reset_index(drop=True)

df_raw

,AGE_SURGERY,SEX,FHX,FHX_1DEGREE,TOBACCO,BMI,PHX_ABDOMINAL_SURGERY,PHX_PULM,PHX_CVD,ATCD_COLONO,...,INVOLVEMENT_DISTAL.MARGIN,SAFETY.MARGIN_PROXIMAL,SAFETY.MARGIN_DISTAL,INVASION_PERINEURAL,INVASION_VASC,INVASION_LYMPH,PATH_TUMOR.PERFORATION,SUPP_N0_N+,NUM_LN_ALL,NUM_LN_POS
0,61,F,0,0.0,1.0,16.23,1.0,0,1,1.0,...,1,19.0,1.0,2.0,1.0,1.0,2.0,N0,19,0.0
1,70,H,0,0.0,0.0,27.68,0.0,0,1,0.0,...,1,14.2,0.5,2.0,1.0,1.0,1.0,N1a,23,1.0
2,39,H,0,0.0,1.0,20.23,0.0,1,1,0.0,...,1,13.0,2.8,1.0,1.0,1.0,1.0,N1a,40,1.0
3,38,H,1,0.0,1.0,20.30,0.0,1,0,0.0,...,1,14.0,16.0,2.0,2.0,1.0,1.0,NaN,25,6.0
4,25,F,0,0.0,0.0,17.30,1.0,0,0,0.0,...,2,NaN,NaN,1.0,1.0,1.0,1.0,N1b,35,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,73,H,0,0.0,0.0,NaN,0.0,0,1,0.0,...,1,5.0,11.5,1.0,2.0,2.0,1.0,N0,32,0.0
222,55,H,0,0.0,1.0,25.73,0.0,1,1,0.0,...,1,9.0,10.0,1.0,2.0,1.0,1.0,N0,32,0.0
223,58,H,0,NaN,1.0,21.60,0.0,1,1,0.0,...,1,5.8,16.5,1.0,2.0,1.0,1.0,N0,51,0.0
224,82,H,0,0.0,1.0,NaN,1.0,1,1,1.0,...,1,6.5,18.0,2.0,1.0,2.0,1.0,N0,30,0.0


In [5]:
# transform to int COMPLICATION
df_raw[selected_target] = df_raw[selected_target].astype(int)


In [6]:
X = df_raw[selected_inputs]
y = df_raw[selected_target].values.ravel().astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

In [7]:
model_configs = {
    "Random Forest": {
        "model": RandomForestClassifier(),
        "params": {
            "clf__n_estimators": [50, 100],
            "clf__max_depth": [None, 10, 20],
        },
        "requires_encoding": False
    },
    "Logistic Regression": {
        "model": LogisticRegression(max_iter=1000, class_weight="balanced"),
        "params": {
            "clf__C": [0.01, 0.1, 1.0, 10.0]
        },
        "requires_encoding": True
    },
    "SVM": {
        "model": SVC(probability=True, class_weight="balanced"),
        "params": {
            "clf__C": [0.1, 1, 10],
            "clf__kernel": ["rbf", "linear"]
        },
        "requires_encoding": True
    },
    "Gradient Boosting": {
        "model": GradientBoostingClassifier(),
        "params": {
            "clf__n_estimators": [50, 100],
            "clf__learning_rate": [0.05, 0.1]
        },
        "requires_encoding": False
    }
}

In [ ]:
results = []
pipelines = {}

for name, config in model_configs.items():
    print(f"Training {name}...")

    cat_cols = X_train.select_dtypes(include="object").columns

    if config["requires_encoding"]:
        preprocessor = ColumnTransformer([
            ("cat", Pipeline([
                ("imputer", SimpleImputer(strategy="most_frequent")),
                ("encoder", OneHotEncoder(handle_unknown="ignore"))
            ]), cat_cols)
        ], remainder="passthrough")
    else:
        preprocessor = ColumnTransformer([
            ("cat", Pipeline([
                ("imputer", SimpleImputer(strategy="most_frequent")),
                ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1))
            ]), cat_cols)
        ], remainder="passthrough")

    pipeline = Pipeline([
        ("preprocessing", preprocessor),
        ("clf", config["model"])
    ])

    grid = GridSearchCV(pipeline, config["params"], cv=10, scoring="recall_macro", n_jobs=-1)
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)

    recall = recall_score(y_test, y_pred, average="macro", zero_division=0)
    results.append((name, recall, grid.best_estimator_))
    pipelines[name] = grid.best_estimator_

# Sort and select best
results.sort(key=lambda x: x[1], reverse=True)
best_model_name, best_recall, best_pipeline = results[0]

print(f"\n✅ Best model: {best_model_name} with recall score: {best_recall:.4f}")

Training Random Forest...
Training Logistic Regression...
Training SVM...
Training Gradient Boosting...

✅ Best model: Logistic Regression with F1 score: 0.5931
